# Run various tests on models

In [1]:
from game.game import Game
from game.data.generator import create_numbers
import torch
import os

In [2]:
# loads checkpoint and does one epoch, shows acc on validation set
def load(last_save, model_size, model_number, run_number, vocab, sender, receiver, embedding, length=1):
    epochs = last_save + 1
    save_name = "game/models/game{}_{}_{}_{}_{}_{}_{}".format(model_size, model_number, run_number, vocab, sender, receiver, embedding)
    if(length > 1):
        save_name = "{}_{}".format(save_name, length)
    

    params = [
        '--batch_size=1',
        '--n_epochs={}'.format(epochs),
        '--vocab_size={}'.format(vocab),
        '--sender_hidden={}'.format(sender),
        '--receiver_hidden={}'.format(receiver),
        '--lr=0.001',
        '--max_len={}'.format(length),
        '--sender_entropy=0.5',
        '--sender_embedding={}'.format(embedding),
        '--receiver_embedding={}'.format(embedding),
        '--validation_freq=10',
        '--train_data=game/data/train10.txt',
        '--validation_data=game/data/validation10.txt',
        '--load_from_checkpoint={}_{}.tar'.format(save_name, last_save)
    ]
    game = Game(params)
    game.play()
    return game

In [3]:
def predict_and_print(model, inputs, labels):
    inputs_tensor = torch.FloatTensor(inputs)
    labels_tensor = torch.FloatTensor(labels)
    prediction = model.game.forward(inputs_tensor,labels_tensor)
    messages = prediction[1].message.tolist()
    outputs = prediction[1].receiver_output.tolist()
    for q in range(len(outputs)):
        print("{} {} {} {}".format(inputs[q], messages[q], outputs[q], labels[q]))
    unique_outputs = set([item[0] for item in outputs])
    number_unique = len(unique_outputs)
    acc = prediction[1].aux['acc'].mean().item()
    print("Unique outputs: {}: {}".format(number_unique, unique_outputs))
    print("Accuracy: {}".format(acc))
    return acc, number_unique, outputs


In [4]:
extra_tests = {}
def test_above_limit(game, start, end, number_of_samples):
    # creates more samples for model to predict
    inputs, labels = extra_tests.get((start, end, number_of_samples), (None, None))
    if(inputs is None or labels is None):
        inputs, labels = create_numbers(end, number_of_samples, False, False, start)
        extra_tests[(start, end, number_of_samples)] = (inputs, labels)
        print("Inputs: {}".format(inputs))
        print("Labels: {}".format(labels))
    print("Testing {} samples of numbers that sum up to [{} - {}]".format(number_of_samples, start, end))
    return predict_and_print(game, inputs, labels)

In [5]:
def test_all_set(game, n):
    inputs = []
    labels = []
    for i in range(n+1):
        for j in range(n+1):
            if i+j <= n:
                inputs.append([i, j])
                labels.append([i+j]) 
    print("Testing {} samples of numbers that sum up to {}".format(len(inputs), n))
    return predict_and_print(game, inputs, labels)

In [6]:
def test_validation_only(game):
    frame = game.trainer.validation_data.dataset.frame
    inputs = [f[0].tolist() for f in frame]
    labels = [f[1].tolist() for f in frame]
    return predict_and_print(game, inputs, labels)

In [7]:
def test_some_other_numbers(game):
    inputs = [[9.0, 0.2], [0., 0.3],[6.0, 4.0],[6.0, 4.0],[9.0, 1.0],[9.0, 1.0],[9.0, 1.0],[1.0, 1.0],[1.0, 1.0],[1.0, 1.0], [0.0, 1.0], [0.0, 0.0], [-1.0, 4.0], [5.0, -2.0], [-1.0, 0.0], [-2.0, -2.0], [-9.0, -1.0], [8.0, -3.0], [-10.0, -10.0]]
    labels = [[9.2],[.3],[10.],[10.],[10.],[10.],[10.],[2.0],[2.0],[2.0],[1.0],[0.0],[3.0],[3.0], [-1.0], [-4.0], [-10.0], [5.0], [-20.0]]
    return predict_and_print(game, inputs, labels)

In [8]:
def test_synonymy(game, n, repeat):
    limit = n+1
    accs_input = {}
    messages_input = {}
    accs_label = {}
    messages_label = {}
    for i in range(limit):
        for j in range(limit):
            if(i+j <= n):
                inputs = []
                labels = []
                for p in range(repeat):
                    inputs.append([i, j])
                    labels.append([i+j]) 
                
                a, m, o = predict_and_print(game, inputs, labels)
                
                input = (inputs[0][0],inputs[0][1])
                label = labels[0][0]
                accs_input[input] = a
                messages_input[input] = m
                
                l = messages_label.get(label, [])
                l.append(o)
                messages_label[label] = l
                
                l = accs_label.get(label, [])
                l.append(a)
                accs_label[label] = l
                
    for i in range(limit):
        outputs = [output[0] for outputs in messages_label[i] for output in outputs]
        messages_label[i] = len(set(outputs))
        accs_label[i] = sum(accs_label[i])/len(accs_label[i])
        
    print("Unique messages per input: {}".format(messages_input))
    print("Accuracies per input: {}".format(accs_input))
    return accs_label, messages_label, _

In [9]:
def test_various_ranges(game, start_numbers, end_numbers, n_samples):
    accs = []
    ns_symbols = []
    for i in range(len(start_numbers)):  
        acc, n_symbols, _ = test_above_limit(game, start_numbers[i], end_numbers[i], n_samples[i])
        accs.append(acc)
        ns_symbols.append(n_symbols)
    print("Accuracies: {}".format(accs))
    print("Number of messages: {}".format(ns_symbols))
    return accs, ns_symbols, _

In [10]:
def run_test(games, test):
    accs = []
    ns_symbols= []
    
    for i in range(len(games)):
        print("\n--------")
        print("Running test for game {}".format(i))
        acc, n_symbols, _ = test(games[i])
        accs.append(acc)
        ns_symbols.append(n_symbols)
        
    print("\n--------")
    print("All games:")
    for i in range(len(accs)):
        print("Game {},  Number of messages: {}, Accuracy: {}".format(i, ns_symbols[i], accs[i]))

In [11]:
games10 = [
    load(1500, 10, 14, 1, 32, 64, 32, 7), # game10_14_1_32_64_32_7_1500.tar
    load(1500, 10, 14, 1, 64, 128, 64, 8), # game10_14_1_64_128_64_8_1500.tar
    load(1500, 10, 14, 1, 32, 32, 32, 7), #game10_14_1_32_32_32_7_1500.tar
    load(1500, 10, 14, 1, 64, 64, 64, 7), #game10_14_1_64_64_64_7_1500.tar
    load(1500, 10, 15, 1, 64, 64, 64, 7, 2), #game10_15_1_64_64_64_7_2_1500.tar
    load(6000, 10, 15, 2, 16, 16, 16, 5, 2)
]

Preparing dataset...
Samples in training set: 52
Samples in test set: 14

----------
Building model...
# Initializing model, trainer, and optimizer from game/models/game10_14_1_32_64_32_7_1500.tar
# loading trainer state from game/models/game10_14_1_32_64_32_7_1500.tar
SenderReceiverRnnReinforce(
  (sender): RnnSenderReinforce(
    (agent): LayerWrapper(
      (output): Linear(in_features=2, out_features=64, bias=True)
    )
    (hidden_to_output): Linear(in_features=64, out_features=32, bias=True)
    (embedding): Embedding(32, 7)
    (cells): ModuleList(
      (0): RNNCell(7, 64)
    )
  )
  (receiver): RnnReceiverDeterministic(
    (agent): LayerWrapper(
      (output): Linear(in_features=32, out_features=1, bias=True)
    )
    (encoder): RnnEncoder(
      (cell): RNN(7, 32, batch_first=True)
      (embedding): Embedding(32, 7)
    )
  )
  (mechanics): CommunicationRnnReinforce()
)
Number of parameters 8744

----------
Start of training... 2021-11-11 15:15:22.984955
{"loss": -0.275

# Test whole number range

In [12]:
run_test(games10, lambda x: test_all_set(x, 10))


--------
Running test for game 0
Testing 66 samples of numbers that sum up to 10
[0, 0] [21, 0] [1.2115983963012695] [0]
[0, 1] [21, 0] [1.2115983963012695] [1]
[0, 2] [21, 0] [1.2115983963012695] [2]
[0, 3] [30, 0] [3.077895164489746] [3]
[0, 4] [3, 0] [4.076868534088135] [4]
[0, 5] [9, 0] [5.006210803985596] [5]
[0, 6] [19, 0] [5.893954277038574] [6]
[0, 7] [12, 0] [6.896281719207764] [7]
[0, 8] [25, 0] [7.978322505950928] [8]
[0, 9] [17, 0] [9.262351036071777] [9]
[0, 10] [13, 0] [9.914275169372559] [10]
[1, 0] [21, 0] [1.2115983963012695] [1]
[1, 1] [21, 0] [1.2115983963012695] [2]
[1, 2] [30, 0] [3.077895164489746] [3]
[1, 3] [20, 0] [4.0557475090026855] [4]
[1, 4] [9, 0] [5.006210803985596] [5]
[1, 5] [8, 0] [5.9192280769348145] [6]
[1, 6] [27, 0] [6.989236831665039] [7]
[1, 7] [23, 0] [7.969519138336182] [8]
[1, 8] [28, 0] [9.256339073181152] [9]
[1, 9] [29, 0] [9.278679847717285] [10]
[2, 0] [21, 0] [1.2115983963012695] [2]
[2, 1] [30, 0] [3.077895164489746] [3]
[2, 2] [20, 0]

# Test validation only

In [13]:
run_test(games10, lambda x: test_validation_only(x))


--------
Running test for game 0
[5.0, 3.0] [16, 0] [8.12182903289795] [8.0]
[5.0, 2.0] [6, 0] [6.977642059326172] [7.0]
[0.0, 10.0] [13, 0] [9.914275169372559] [10.0]
[7.0, 0.0] [12, 0] [6.896281719207764] [7.0]
[5.0, 0.0] [9, 0] [5.006210803985596] [5.0]
[4.0, 0.0] [20, 0] [4.0557475090026855] [4.0]
[0.0, 5.0] [31, 0] [4.06857967376709] [5.0]
[7.0, 3.0] [13, 0] [9.914275169372559] [10.0]
[1.0, 2.0] [30, 0] [3.077895164489746] [3.0]
[1.0, 8.0] [17, 0] [9.262351036071777] [9.0]
[2.0, 8.0] [28, 0] [9.256339073181152] [10.0]
[2.0, 0.0] [21, 0] [1.2115983963012695] [2.0]
[4.0, 3.0] [6, 0] [6.977642059326172] [7.0]
[5.0, 1.0] [19, 0] [5.893954277038574] [6.0]
Unique outputs: 12: {1.2115983963012695, 3.077895164489746, 4.0557475090026855, 5.006210803985596, 6.977642059326172, 6.896281719207764, 8.12182903289795, 9.914275169372559, 4.06857967376709, 9.262351036071777, 9.256339073181152, 5.893954277038574}
Accuracy: 0.7857142686843872

--------
Running test for game 1
[5.0, 3.0] [56, 0] [8.1

# Test above number range

In [14]:
start_numbers = [11, 13, 50, 101]
end_numbers = [12, 20, 100, 100000]
n_samples = [10, 100, 100, 1000]
run_test(games10, lambda x: test_various_ranges(x, start_numbers, end_numbers, n_samples))


--------
Running test for game 0
Generating 10 samples...
Inputs: [[4, 7], [5, 6], [2, 10], [11, 1], [1, 10], [6, 5], [8, 4], [10, 1], [7, 4], [0, 11]]
Labels: [[11], [11], [12], [12], [11], [11], [12], [11], [11], [11]]
Testing 10 samples of numbers that sum up to [11 - 12]
[4, 7] [29, 0] [9.278679847717285] [11]
[5, 6] [13, 0] [9.914275169372559] [11]
[2, 10] [1, 0] [9.262864112854004] [12]
[11, 1] [13, 0] [9.914275169372559] [12]
[1, 10] [26, 0] [9.264168739318848] [11]
[6, 5] [29, 0] [9.278679847717285] [11]
[8, 4] [1, 0] [9.262864112854004] [12]
[10, 1] [13, 0] [9.914275169372559] [11]
[7, 4] [17, 0] [9.262351036071777] [11]
[0, 11] [13, 0] [9.914275169372559] [11]
Unique outputs: 5: {9.278679847717285, 9.264168739318848, 9.262351036071777, 9.914275169372559, 9.262864112854004}
Accuracy: 0.0
Generating 100 samples...
Inputs: [[12, 2], [14, 0], [0, 20], [16, 3], [2, 18], [12, 7], [12, 8], [2, 13], [2, 11], [7, 9], [2, 12], [11, 4], [8, 9], [10, 10], [7, 11], [17, 2], [11, 2], [4, 

[16811, 37368] [13, 0] [9.914275169372559] [54179]
[25112, 49582] [13, 0] [9.914275169372559] [74694]
[35098, 43773] [13, 0] [9.914275169372559] [78871]
[7225, 62487] [13, 0] [9.914275169372559] [69712]
[15153, 50647] [13, 0] [9.914275169372559] [65800]
[79330, 7180] [13, 0] [9.914275169372559] [86510]
[21093, 60616] [13, 0] [9.914275169372559] [81709]
[63879, 26935] [13, 0] [9.914275169372559] [90814]
[10820, 2871] [13, 0] [9.914275169372559] [13691]
[35262, 40594] [13, 0] [9.914275169372559] [75856]
[2878, 21292] [13, 0] [9.914275169372559] [24170]
[16620, 15745] [13, 0] [9.914275169372559] [32365]
[14347, 879] [13, 0] [9.914275169372559] [15226]
[45653, 48396] [13, 0] [9.914275169372559] [94049]
[13905, 82334] [13, 0] [9.914275169372559] [96239]
[19597, 41730] [13, 0] [9.914275169372559] [61327]
[39138, 35605] [13, 0] [9.914275169372559] [74743]
[33185, 7583] [13, 0] [9.914275169372559] [40768]
[6141, 62780] [13, 0] [9.914275169372559] [68921]
[14084, 50361] [13, 0] [9.9142751693725

[29482, 62965] [13, 0] [9.914275169372559] [92447]
[51724, 41033] [13, 0] [9.914275169372559] [92757]
[8077, 7270] [13, 0] [9.914275169372559] [15347]
[34573, 909] [13, 0] [9.914275169372559] [35482]
[74463, 21583] [13, 0] [9.914275169372559] [96046]
[2264, 5554] [13, 0] [9.914275169372559] [7818]
[34010, 27960] [13, 0] [9.914275169372559] [61970]
[16448, 30927] [13, 0] [9.914275169372559] [47375]
[16071, 11924] [13, 0] [9.914275169372559] [27995]
[31423, 22702] [13, 0] [9.914275169372559] [54125]
[33996, 10987] [13, 0] [9.914275169372559] [44983]
[9940, 8172] [13, 0] [9.914275169372559] [18112]
[38827, 29316] [17, 0] [9.262351036071777] [68143]
[9512, 8375] [13, 0] [9.914275169372559] [17887]
[6075, 4110] [13, 0] [9.914275169372559] [10185]
[15183, 11393] [17, 0] [9.262351036071777] [26576]
[25929, 25124] [13, 0] [9.914275169372559] [51053]
[3303, 32983] [13, 0] [9.914275169372559] [36286]
[79021, 14027] [13, 0] [9.914275169372559] [93048]
[6477, 6218] [13, 0] [9.914275169372559] [126

[2, 10] [2, 0] [9.366336822509766] [12]
[11, 1] [5, 0] [9.612194061279297] [12]
[1, 10] [63, 0] [9.633731842041016] [11]
[6, 5] [32, 0] [9.569334030151367] [11]
[8, 4] [2, 0] [9.366336822509766] [12]
[10, 1] [2, 0] [9.366336822509766] [11]
[7, 4] [41, 0] [9.621837615966797] [11]
[0, 11] [2, 0] [9.366336822509766] [11]
Unique outputs: 6: {9.366336822509766, 9.633731842041016, 9.569334030151367, 9.629152297973633, 9.612194061279297, 9.621837615966797}
Accuracy: 0.0
Testing 100 samples of numbers that sum up to [13 - 20]
[12, 2] [35, 0] [9.449699401855469] [14]
[14, 0] [18, 0] [9.629152297973633] [14]
[0, 20] [41, 0] [9.621837615966797] [20]
[16, 3] [2, 0] [9.366336822509766] [19]
[2, 18] [2, 0] [9.366336822509766] [20]
[12, 7] [41, 0] [9.621837615966797] [19]
[12, 8] [36, 0] [9.559636116027832] [20]
[2, 13] [2, 0] [9.366336822509766] [15]
[2, 11] [35, 0] [9.449699401855469] [13]
[7, 9] [41, 0] [9.621837615966797] [16]
[2, 12] [2, 0] [9.366336822509766] [14]
[11, 4] [18, 0] [9.62915229797

[3659, 91710] [36, 0] [9.559636116027832] [95369]
[62078, 12003] [41, 0] [9.621837615966797] [74081]
[68804, 14335] [2, 0] [9.366336822509766] [83139]
[50736, 35708] [18, 0] [9.629152297973633] [86444]
[12196, 68090] [18, 0] [9.629152297973633] [80286]
[27001, 16354] [36, 0] [9.559636116027832] [43355]
[28144, 31059] [2, 0] [9.366336822509766] [59203]
[47869, 869] [63, 0] [9.633731842041016] [48738]
[28856, 43829] [16, 0] [8.854222297668457] [72685]
[8069, 29019] [25, 0] [9.612892150878906] [37088]
[74539, 14852] [35, 0] [9.449699401855469] [89391]
[12356, 27494] [2, 0] [9.366336822509766] [39850]
[8071, 12108] [2, 0] [9.366336822509766] [20179]
[12726, 12022] [18, 0] [9.629152297973633] [24748]
[11871, 50390] [63, 0] [9.633731842041016] [62261]
[20159, 19905] [2, 0] [9.366336822509766] [40064]
[52430, 6786] [32, 0] [9.569334030151367] [59216]
[6280, 6556] [2, 0] [9.366336822509766] [12836]
[5372, 11745] [18, 0] [9.629152297973633] [17117]
[25139, 19404] [32, 0] [9.569334030151367] [44

...

[8240, 89440] [34, 12, 0] [10.027578353881836] [97680]
[13498, 20278] [18, 6, 0] [10.022462844848633] [33776]
[21029, 2969] [2, 10, 0] [10.028325080871582] [23998]
[355, 2765] [12, 59, 0] [10.02041244506836] [3120]
[28891, 58485] [12, 25, 0] [10.02871036529541] [87376]
[17505, 39986] [32, 5, 0] [10.022119522094727] [57491]
[8129, 25858] [41, 54, 0] [10.036773681640625] [33987]
[8036, 55782] [45, 4, 0] [10.045374870300293] [63818]
[60028, 35592] [44, 44, 0] [10.015201568603516] [95620]
[61375, 16853] [12, 41, 0] [10.030403137207031] [78228]
[12017, 36838] [13, 10, 0] [10.023561477661133] [48855]
[31228, 52936] [60, 12, 0] [10.023394584655762] [84164]
[1583, 32416] [54, 18, 0] [10.032110214233398] [33999]
[7611, 7366] [12, 18, 0] [10.029339790344238] [14977]
[17460, 28803] [53, 58, 0] [10.029549598693848] [46263]
[34201, 22309] [58, 5, 0] [10.017181396484375] [56510]
[9305, 20239] [2, 3, 0] [10.034660339355469] [29544]
[46487, 53390] [3, 54, 0] [10.036550521850586] [99877]
[15039, 11502]

[33891, 48625] [62, 43, 0] [10.039138793945312] [82516]
[15032, 49422] [53, 43, 0] [10.03108024597168] [64454]
[32950, 1746] [54, 59, 0] [10.021821975708008] [34696]
[5154, 76804] [10, 44, 0] [10.023683547973633] [81958]
[12864, 32033] [15, 10, 0] [10.024216651916504] [44897]
[33897, 32525] [43, 43, 0] [10.04295539855957] [66422]
[45015, 20430] [59, 60, 0] [10.010764122009277] [65445]
[37030, 7488] [59, 4, 0] [10.029282569885254] [44518]
[32031, 9209] [3, 37, 0] [10.034764289855957] [41240]
[69377, 3988] [32, 37, 0] [10.032499313354492] [73365]
[5653, 30273] [18, 3, 0] [10.02686882019043] [35926]
[8957, 85121] [12, 12, 0] [10.035079956054688] [94078]
[42338, 28348] [45, 2, 0] [10.030366897583008] [70686]
[27434, 31022] [61, 16, 0] [10.043496131896973] [58456]
[34105, 44546] [20, 43, 0] [10.0376615524292] [78651]
[22009, 8898] [60, 44, 0] [10.01957893371582] [30907]
[34335, 40895] [12, 10, 0] [10.033077239990234] [75230]
[43013, 39670] [43, 16, 0] [10.039999961853027] [82683]
[464, 4617

[12, 2] [1, 1, 0] [10.036968231201172] [14]
[14, 0] [1, 15, 0] [10.018472671508789] [14]
[0, 20] [1, 11, 0] [10.004566192626953] [20]
[16, 3] [15, 1, 0] [9.979684829711914] [19]
[2, 18] [1, 13, 0] [10.02072811126709] [20]
[12, 7] [1, 1, 0] [10.036968231201172] [19]
[12, 8] [1, 13, 0] [10.02072811126709] [20]
[2, 13] [1, 6, 0] [10.03327751159668] [15]
[2, 11] [1, 12, 0] [9.995586395263672] [13]
[7, 9] [14, 6, 0] [9.971052169799805] [16]
[2, 12] [1, 6, 0] [10.03327751159668] [14]
[11, 4] [1, 9, 0] [9.99515151977539] [15]
[8, 9] [1, 1, 0] [10.036968231201172] [17]
[10, 10] [1, 14, 0] [10.033441543579102] [20]
[7, 11] [1, 13, 0] [10.02072811126709] [18]
[17, 2] [6, 6, 0] [9.980009078979492] [19]
[11, 2] [14, 1, 0] [9.982091903686523] [13]
[4, 13] [1, 6, 0] [10.03327751159668] [17]
[3, 11] [14, 13, 0] [9.971914291381836] [14]
[1, 19] [1, 8, 0] [9.992754936218262] [20]
[5, 14] [1, 1, 0] [10.036968231201172] [19]
[12, 1] [6, 1, 0] [9.98228645324707] [13]
[10, 8] [13, 1, 0] [9.97714614868164] 

[37973, 51864] [1, 12, 0] [9.995586395263672] [89837]
[33410, 506] [1, 15, 0] [10.018472671508789] [33916]
[14334, 14175] [14, 15, 0] [10.000364303588867] [28509]
[15419, 10763] [1, 13, 0] [10.02072811126709] [26182]
[18634, 81031] [1, 2, 0] [9.98447036743164] [99665]
[7295, 58273] [1, 13, 0] [10.02072811126709] [65568]
[32645, 2599] [1, 9, 0] [9.99515151977539] [35244]
[53774, 30503] [1, 15, 0] [10.018472671508789] [84277]
[72163, 14971] [1, 13, 0] [10.02072811126709] [87134]
[33317, 1080] [1, 2, 0] [9.98447036743164] [34397]
[1378, 10344] [1, 1, 0] [10.036968231201172] [11722]
[34060, 49109] [6, 15, 0] [10.006284713745117] [83169]
[41271, 13826] [15, 1, 0] [9.979684829711914] [55097]
[6760, 3895] [1, 8, 0] [9.992754936218262] [10655]
[5925, 29303] [1, 14, 0] [10.033441543579102] [35228]
[13062, 7081] [1, 15, 0] [10.018472671508789] [20143]
[3763, 48636] [1, 13, 0] [10.02072811126709] [52399]
[19923, 34587] [14, 15, 0] [10.000364303588867] [54510]
[4403, 59270] [6, 14, 0] [9.966828346

[14337, 60734] [1, 6, 0] [10.03327751159668] [75071]
[72214, 7633] [14, 15, 0] [10.000364303588867] [79847]
[18628, 17233] [1, 15, 0] [10.018472671508789] [35861]
[73717, 739] [1, 13, 0] [10.02072811126709] [74456]
[718, 6121] [1, 2, 0] [9.98447036743164] [6839]
[6634, 243] [13, 1, 0] [9.97714614868164] [6877]
[15854, 29199] [1, 1, 0] [10.036968231201172] [45053]
[30420, 51983] [1, 1, 0] [10.036968231201172] [82403]
[855, 4994] [1, 6, 0] [10.03327751159668] [5849]
[52772, 14731] [15, 1, 0] [9.979684829711914] [67503]
[37855, 17926] [1, 6, 0] [10.03327751159668] [55781]
[3171, 20465] [1, 15, 0] [10.018472671508789] [23636]
[85612, 12720] [6, 13, 0] [9.98277473449707] [98332]
[54502, 5586] [1, 15, 0] [10.018472671508789] [60088]
[1066, 4013] [1, 12, 0] [9.995586395263672] [5079]
[3962, 130] [1, 13, 0] [10.02072811126709] [4092]
[51465, 6953] [1, 14, 0] [10.033441543579102] [58418]
[11113, 63041] [1, 12, 0] [9.995586395263672] [74154]
[10163, 42958] [6, 1, 0] [9.98228645324707] [53121]
[1

# Test various numbers

In [15]:
run_test(games10, lambda x: test_some_other_numbers(x))


--------
Running test for game 0
[9.0, 0.2] [1, 0] [9.262864112854004] [9.2]
[0.0, 0.3] [21, 0] [1.2115983963012695] [0.3]
[6.0, 4.0] [13, 0] [9.914275169372559] [10.0]
[6.0, 4.0] [13, 0] [9.914275169372559] [10.0]
[9.0, 1.0] [13, 0] [9.914275169372559] [10.0]
[9.0, 1.0] [13, 0] [9.914275169372559] [10.0]
[9.0, 1.0] [17, 0] [9.262351036071777] [10.0]
[1.0, 1.0] [21, 0] [1.2115983963012695] [2.0]
[1.0, 1.0] [21, 0] [1.2115983963012695] [2.0]
[1.0, 1.0] [21, 0] [1.2115983963012695] [2.0]
[0.0, 1.0] [21, 0] [1.2115983963012695] [1.0]
[0.0, 0.0] [21, 0] [1.2115983963012695] [0.0]
[-1.0, 4.0] [30, 0] [3.077895164489746] [3.0]
[5.0, -2.0] [30, 0] [3.077895164489746] [3.0]
[-1.0, 0.0] [21, 0] [1.2115983963012695] [-1.0]
[-2.0, -2.0] [21, 0] [1.2115983963012695] [-4.0]
[-9.0, -1.0] [21, 0] [1.2115983963012695] [-10.0]
[8.0, -3.0] [9, 0] [5.006210803985596] [5.0]
[-10.0, -10.0] [21, 0] [1.2115983963012695] [-20.0]
Unique outputs: 6: {1.2115983963012695, 3.077895164489746, 5.006210803985596, 9.

# Test synonymy

In [16]:
number_range = 10
repeat_times = 100
run_test(games10, lambda x: test_synonymy(x, 10, 100))


--------
Running test for game 0
[0, 0] [21, 0] [1.2115983963012695] [0]
[0, 0] [21, 0] [1.2115983963012695] [0]
[0, 0] [21, 0] [1.2115983963012695] [0]
[0, 0] [21, 0] [1.2115983963012695] [0]
[0, 0] [21, 0] [1.2115983963012695] [0]
[0, 0] [21, 0] [1.2115983963012695] [0]
[0, 0] [21, 0] [1.2115983963012695] [0]
[0, 0] [21, 0] [1.2115983963012695] [0]
[0, 0] [21, 0] [1.2115983963012695] [0]
[0, 0] [21, 0] [1.2115983963012695] [0]
[0, 0] [21, 0] [1.2115983963012695] [0]
[0, 0] [21, 0] [1.2115983963012695] [0]
[0, 0] [21, 0] [1.2115983963012695] [0]
[0, 0] [21, 0] [1.2115983963012695] [0]
[0, 0] [21, 0] [1.2115983963012695] [0]
[0, 0] [21, 0] [1.2115983963012695] [0]
[0, 0] [21, 0] [1.2115983963012695] [0]
[0, 0] [21, 0] [1.2115983963012695] [0]
[0, 0] [21, 0] [1.2115983963012695] [0]
[0, 0] [21, 0] [1.2115983963012695] [0]
[0, 0] [21, 0] [1.2115983963012695] [0]
[0, 0] [21, 0] [1.2115983963012695] [0]
[0, 0] [21, 0] [1.2115983963012695] [0]
[0, 0] [21, 0] [1.2115983963012695] [0]
[0, 0]

[0, 4] [20, 0] [4.0557475090026855] [4]
[0, 4] [3, 0] [4.076868534088135] [4]
[0, 4] [20, 0] [4.0557475090026855] [4]
[0, 4] [31, 0] [4.06857967376709] [4]
[0, 4] [9, 0] [5.006210803985596] [4]
[0, 4] [31, 0] [4.06857967376709] [4]
[0, 4] [20, 0] [4.0557475090026855] [4]
[0, 4] [3, 0] [4.076868534088135] [4]
[0, 4] [20, 0] [4.0557475090026855] [4]
[0, 4] [20, 0] [4.0557475090026855] [4]
[0, 4] [31, 0] [4.06857967376709] [4]
[0, 4] [20, 0] [4.0557475090026855] [4]
[0, 4] [31, 0] [4.06857967376709] [4]
[0, 4] [31, 0] [4.06857967376709] [4]
[0, 4] [20, 0] [4.0557475090026855] [4]
[0, 4] [18, 0] [5.984326362609863] [4]
[0, 4] [20, 0] [4.0557475090026855] [4]
[0, 4] [20, 0] [4.0557475090026855] [4]
[0, 4] [20, 0] [4.0557475090026855] [4]
[0, 4] [3, 0] [4.076868534088135] [4]
[0, 4] [31, 0] [4.06857967376709] [4]
[0, 4] [20, 0] [4.0557475090026855] [4]
[0, 4] [20, 0] [4.0557475090026855] [4]
[0, 4] [20, 0] [4.0557475090026855] [4]
[0, 4] [31, 0] [4.06857967376709] [4]
[0, 4] [31, 0] [4.06857

[0, 10] [13, 0] [9.914275169372559] [10]
[0, 10] [17, 0] [9.262351036071777] [10]
[0, 10] [28, 0] [9.256339073181152] [10]
[0, 10] [17, 0] [9.262351036071777] [10]
[0, 10] [29, 0] [9.278679847717285] [10]
[0, 10] [17, 0] [9.262351036071777] [10]
[0, 10] [13, 0] [9.914275169372559] [10]
[0, 10] [28, 0] [9.256339073181152] [10]
[0, 10] [1, 0] [9.262864112854004] [10]
[0, 10] [17, 0] [9.262351036071777] [10]
[0, 10] [26, 0] [9.264168739318848] [10]
[0, 10] [28, 0] [9.256339073181152] [10]
[0, 10] [13, 0] [9.914275169372559] [10]
[0, 10] [1, 0] [9.262864112854004] [10]
[0, 10] [29, 0] [9.278679847717285] [10]
[0, 10] [17, 0] [9.262351036071777] [10]
[0, 10] [17, 0] [9.262351036071777] [10]
[0, 10] [17, 0] [9.262351036071777] [10]
[0, 10] [26, 0] [9.264168739318848] [10]
[0, 10] [17, 0] [9.262351036071777] [10]
[0, 10] [13, 0] [9.914275169372559] [10]
[0, 10] [13, 0] [9.914275169372559] [10]
[0, 10] [13, 0] [9.914275169372559] [10]
[0, 10] [28, 0] [9.256339073181152] [10]
[0, 10] [1, 0] [9.

[1, 1] [21, 0] [1.2115983963012695] [2]
[1, 1] [21, 0] [1.2115983963012695] [2]
[1, 1] [21, 0] [1.2115983963012695] [2]
[1, 1] [21, 0] [1.2115983963012695] [2]
[1, 1] [21, 0] [1.2115983963012695] [2]
[1, 1] [21, 0] [1.2115983963012695] [2]
[1, 1] [21, 0] [1.2115983963012695] [2]
[1, 1] [21, 0] [1.2115983963012695] [2]
[1, 1] [21, 0] [1.2115983963012695] [2]
[1, 1] [21, 0] [1.2115983963012695] [2]
[1, 1] [21, 0] [1.2115983963012695] [2]
[1, 1] [21, 0] [1.2115983963012695] [2]
[1, 1] [21, 0] [1.2115983963012695] [2]
[1, 1] [21, 0] [1.2115983963012695] [2]
[1, 1] [21, 0] [1.2115983963012695] [2]
[1, 1] [21, 0] [1.2115983963012695] [2]
[1, 1] [21, 0] [1.2115983963012695] [2]
[1, 1] [21, 0] [1.2115983963012695] [2]
[1, 1] [21, 0] [1.2115983963012695] [2]
[1, 1] [21, 0] [1.2115983963012695] [2]
[1, 1] [21, 0] [1.2115983963012695] [2]
[1, 1] [21, 0] [1.2115983963012695] [2]
[1, 1] [21, 0] [1.2115983963012695] [2]
[1, 1] [21, 0] [1.2115983963012695] [2]
[1, 1] [21, 0] [1.2115983963012695] [2]


[1, 5] [8, 0] [5.9192280769348145] [6]
[1, 5] [18, 0] [5.984326362609863] [6]
[1, 5] [18, 0] [5.984326362609863] [6]
[1, 5] [18, 0] [5.984326362609863] [6]
[1, 5] [12, 0] [6.896281719207764] [6]
[1, 5] [15, 0] [5.936023235321045] [6]
[1, 5] [18, 0] [5.984326362609863] [6]
[1, 5] [15, 0] [5.936023235321045] [6]
[1, 5] [18, 0] [5.984326362609863] [6]
[1, 5] [8, 0] [5.9192280769348145] [6]
[1, 5] [8, 0] [5.9192280769348145] [6]
[1, 5] [7, 0] [5.92131233215332] [6]
[1, 5] [18, 0] [5.984326362609863] [6]
[1, 5] [7, 0] [5.92131233215332] [6]
[1, 5] [8, 0] [5.9192280769348145] [6]
[1, 5] [18, 0] [5.984326362609863] [6]
[1, 5] [8, 0] [5.9192280769348145] [6]
[1, 5] [7, 0] [5.92131233215332] [6]
[1, 5] [10, 0] [5.9667229652404785] [6]
[1, 5] [8, 0] [5.9192280769348145] [6]
[1, 5] [8, 0] [5.9192280769348145] [6]
[1, 5] [18, 0] [5.984326362609863] [6]
[1, 5] [19, 0] [5.893954277038574] [6]
[1, 5] [8, 0] [5.9192280769348145] [6]
[1, 5] [19, 0] [5.893954277038574] [6]
[1, 5] [15, 0] [5.936023235321

...

[4, 1] [2, 7, 0] [4.911426544189453] [5]
[4, 1] [11, 3, 0] [4.993143081665039] [5]
[4, 1] [9, 3, 0] [4.9901323318481445] [5]
[4, 1] [12, 3, 0] [4.993558883666992] [5]
[4, 1] [9, 3, 0] [4.9901323318481445] [5]
[4, 1] [8, 3, 0] [5.015501976013184] [5]
[4, 1] [5, 3, 0] [5.003337860107422] [5]
[4, 1] [11, 3, 0] [4.993143081665039] [5]
[4, 1] [8, 3, 0] [5.015501976013184] [5]
[4, 1] [12, 3, 0] [4.993558883666992] [5]
[4, 1] [8, 3, 0] [5.015501976013184] [5]
[4, 1] [4, 9, 0] [4.003976821899414] [5]
[4, 1] [8, 3, 0] [5.015501976013184] [5]
[4, 1] [11, 3, 0] [4.993143081665039] [5]
[4, 1] [5, 3, 0] [5.003337860107422] [5]
[4, 1] [8, 3, 0] [5.015501976013184] [5]
[4, 1] [11, 3, 0] [4.993143081665039] [5]
[4, 1] [12, 3, 0] [4.993558883666992] [5]
[4, 1] [9, 3, 0] [4.9901323318481445] [5]
[4, 1] [8, 3, 0] [5.015501976013184] [5]
[4, 1] [8, 3, 0] [5.015501976013184] [5]
[4, 1] [12, 3, 0] [4.993558883666992] [5]
[4, 1] [9, 3, 0] [4.9901323318481445] [5]
[4, 1] [8, 3, 0] [5.015501976013184] [5]
[4, 

[4, 5] [6, 5, 0] [9.063045501708984] [9]
[4, 5] [13, 13, 0] [9.0099515914917] [9]
[4, 5] [14, 13, 0] [9.971914291381836] [9]
[4, 5] [13, 15, 0] [9.027359008789062] [9]
[4, 5] [15, 15, 0] [9.113143920898438] [9]
[4, 5] [15, 6, 0] [9.039039611816406] [9]
[4, 5] [6, 12, 0] [9.046119689941406] [9]
[4, 5] [15, 14, 0] [9.026142120361328] [9]
[4, 5] [1, 10, 0] [9.011693954467773] [9]
[4, 5] [14, 2, 0] [8.963785171508789] [9]
[4, 5] [5, 1, 0] [9.057090759277344] [9]
[4, 5] [6, 12, 0] [9.046119689941406] [9]
[4, 5] [6, 5, 0] [9.063045501708984] [9]
[4, 5] [1, 10, 0] [9.011693954467773] [9]
[4, 5] [11, 1, 0] [9.055164337158203] [9]
[4, 5] [6, 8, 0] [9.056306838989258] [9]
[4, 5] [14, 8, 0] [9.049470901489258] [9]
[4, 5] [5, 1, 0] [9.057090759277344] [9]
[4, 5] [15, 6, 0] [9.039039611816406] [9]
[4, 5] [15, 6, 0] [9.039039611816406] [9]
[4, 5] [5, 1, 0] [9.057090759277344] [9]
[4, 5] [13, 14, 0] [8.974956512451172] [9]
[4, 5] [12, 1, 0] [9.055135726928711] [9]
[4, 5] [15, 6, 0] [9.039039611816406

[5, 2] [13, 0, 0] [6.999152183532715] [7]
[5, 2] [6, 3, 0] [6.89097785949707] [7]
[5, 2] [9, 8, 0] [7.059845924377441] [7]
[5, 2] [12, 12, 0] [7.04170036315918] [7]
[5, 2] [11, 2, 0] [6.927945137023926] [7]
[5, 2] [11, 2, 0] [6.927945137023926] [7]
[5, 2] [12, 11, 0] [7.063600540161133] [7]
[5, 2] [8, 2, 0] [6.940523147583008] [7]
[5, 2] [8, 9, 0] [7.040762901306152] [7]
[5, 2] [13, 4, 0] [7.024691581726074] [7]
[5, 2] [11, 13, 0] [8.03825855255127] [7]
[5, 2] [15, 7, 0] [7.053105354309082] [7]
[5, 2] [6, 3, 0] [6.89097785949707] [7]
[5, 2] [13, 10, 0] [6.943094253540039] [7]
[5, 2] [12, 12, 0] [7.04170036315918] [7]
[5, 2] [11, 5, 0] [7.057948112487793] [7]
[5, 2] [6, 3, 0] [6.89097785949707] [7]
[5, 2] [15, 7, 0] [7.053105354309082] [7]
[5, 2] [6, 3, 0] [6.89097785949707] [7]
[5, 2] [12, 8, 0] [7.0667219161987305] [7]
[5, 2] [14, 3, 0] [6.873748779296875] [7]
[5, 2] [13, 10, 0] [6.943094253540039] [7]
[5, 2] [13, 7, 0] [6.99271297454834] [7]
[5, 2] [12, 14, 0] [8.03065013885498] [7]


[7, 0] [15, 0, 0] [7.035676002502441] [7]
[7, 0] [15, 7, 0] [7.053105354309082] [7]
[7, 0] [12, 5, 0] [7.059150695800781] [7]
[7, 0] [5, 9, 0] [7.037799835205078] [7]
[7, 0] [12, 8, 0] [7.0667219161987305] [7]
[7, 0] [8, 8, 0] [7.0786638259887695] [7]
[7, 0] [8, 12, 0] [7.052382469177246] [7]
[7, 0] [9, 2, 0] [6.917192459106445] [7]
[7, 0] [5, 9, 0] [7.037799835205078] [7]
[7, 0] [5, 12, 0] [7.049352645874023] [7]
[7, 0] [5, 9, 0] [7.037799835205078] [7]
[7, 0] [12, 8, 0] [7.0667219161987305] [7]
[7, 0] [15, 4, 0] [7.086996078491211] [7]
[7, 0] [2, 14, 0] [7.099540710449219] [7]
[7, 0] [15, 7, 0] [7.053105354309082] [7]
[7, 0] [12, 5, 0] [7.059150695800781] [7]
[7, 0] [13, 10, 0] [6.943094253540039] [7]
[7, 0] [15, 0, 0] [7.035676002502441] [7]
[7, 0] [14, 3, 0] [6.873748779296875] [7]
[7, 0] [15, 10, 0] [6.9311933517456055] [7]
[7, 0] [6, 3, 0] [6.89097785949707] [7]
[7, 0] [2, 13, 0] [7.050071716308594] [7]
[7, 0] [14, 3, 0] [6.873748779296875] [7]
[7, 0] [9, 8, 0] [7.059845924377441

[8, 2] [13, 1, 0] [9.97714614868164] [10]
[8, 2] [14, 6, 0] [9.971052169799805] [10]
[8, 2] [13, 1, 0] [9.97714614868164] [10]
[8, 2] [14, 13, 0] [9.971914291381836] [10]
[8, 2] [14, 15, 0] [10.000364303588867] [10]
[8, 2] [1, 14, 0] [10.033441543579102] [10]
[8, 2] [14, 13, 0] [9.971914291381836] [10]
[8, 2] [1, 5, 0] [10.00227165222168] [10]
[8, 2] [6, 15, 0] [10.006284713745117] [10]
[8, 2] [6, 14, 0] [9.966828346252441] [10]
[8, 2] [15, 1, 0] [9.979684829711914] [10]
[8, 2] [1, 15, 0] [10.018472671508789] [10]
[8, 2] [6, 14, 0] [9.966828346252441] [10]
[8, 2] [14, 13, 0] [9.971914291381836] [10]
[8, 2] [13, 1, 0] [9.97714614868164] [10]
[8, 2] [13, 1, 0] [9.97714614868164] [10]
[8, 2] [1, 9, 0] [9.99515151977539] [10]
[8, 2] [6, 14, 0] [9.966828346252441] [10]
[8, 2] [1, 13, 0] [10.02072811126709] [10]
[8, 2] [14, 1, 0] [9.982091903686523] [10]
[8, 2] [15, 1, 0] [9.979684829711914] [10]
[8, 2] [6, 13, 0] [9.98277473449707] [10]
[8, 2] [13, 1, 0] [9.97714614868164] [10]
[8, 2] [1, 1